# Modeling

Vamos realizar uma modelagem dos dados com algoritmos de machine learning.

## 1. Leitura dos dados

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
#Setando diretorio Data como o atual
os.chdir('../Data')

#Leitura dos dados
# df_parkinson = pd.read_csv('parkinson_normalizado_ss.csv',index_col='name')
# df_parkinson = pd.read_csv('parkinson_normalizado_mm.csv',index_col='name')
df_parkinson = pd.read_csv('parkinson_pca.csv',index_col='name')

#Mapeando dados para conseguir calcular o AUC
df_parkinson['drug'] = df_parkinson['drug'].map({'CBD':1,'placebo':0})

# 2. Modelagem

## 2.1 Classificação

### 2.1.1 Sem variáveis categórias

In [ ]:
def classifier(X,Y,split=1):
    
    #Pessoas do dataset
    unique_people = X.index.unique()

    #Listas para armazenar os resultados finais
    list_score = []
    list_sens = []
    list_spec = []
    list_auc = []
    list_error = []

    for i in range(500):
        if(split == 1):
            #Sample das pessoas
            people_train = np.random.choice(unique_people,size=11,replace=False)
            people_test = unique_people[~unique_people.isin(people_train)]
    
            #Criando treino e test
            X_train = X.loc[people_train]
            Y_train = Y.loc[people_train]
            X_test = X.loc[people_test]
            Y_test = Y.loc[people_test]
        elif(split == 2):
            X_train,X_test,Y_train,Y_test = train_test_split(X,Y,stratify=Y,test_size=0.4)

        #Criando classificador
        dt = DecisionTreeClassifier().fit(X_train,Y_train)

        #Matrix de confusao
        tn, fp, fn, tp = confusion_matrix(Y_test,dt.predict(X_test)).ravel()

        #Sensibilidade
        sens = tp/(tp+fn)
        
        #Especificidade
        spec = tn/(tn+fp)

        #Score
        score = (tn+tp)/(tn+tp+fn+fp)
        
        #AUC
        auc = roc_auc_score(Y_test,dt.predict(X_test))
        
        #Pessoas que erraram
        error = (dt.predict(X_test) == Y_test)
        list_error = np.append(list_error,error[error == False].index.unique().values)
        list_score.append(score)
        list_sens.append(sens)
        list_spec.append(spec)
        list_auc.append(auc)

    print('AUC:',np.nanmean(list_auc))
    print('Score:',np.nanmean(list_score))
    print('Sensibilidade:',np.nanmean(list_sens))
    print('Especificidade:',np.nanmean(list_spec),'\n')
    
    #Calculo do erro por pessoa
    error = pd.Series(list_error).value_counts()
    print(error/error.values.sum())

In [ ]:
#df_parkinson = df_parkinson.loc[(df_parkinson['measure'] == 'P') & (df_parkinson['evaluate'] == 1),['PC4','PC5','drug']]

In [ ]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float')
Y = df_parkinson['drug']

#Classificacao
classifier(X,Y,split=2)

### 2.1.2 Variáveis categórias sem ser dummy

In [ ]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Adicionando variaveis categoricas
X['measure'] = LabelEncoder().fit_transform(df_parkinson['measure'])
X['evaluate'] = df_parkinson['evaluate']

#Classificacao
classifier(X,Y)

### 2.1.3 Variáveis categórias sendo dummy

In [ ]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()
Y = df_parkinson['drug']

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'],prefix='measure'),pd.get_dummies(df_parkinson['evaluate'],prefix='evaluate')],axis=1)

#Classificacao
classifier(X,Y)

### 2.1.3 Váriveis categóricas sendo dummy, porém unidas

In [ ]:
#Separando dataset em variaveis dependentes e independentes
X = df_parkinson.select_dtypes('float').copy()

#Criando dummies
X = pd.concat([X,pd.get_dummies(df_parkinson['measure'] + '_' + df_parkinson['evaluate'].map(lambda x: str(x)))],axis=1)

#Classificacao
classifier(X,Y)

## 2.2 Clusterização

In [ ]:
#Divisão dataset
X = df_parkinson.select_dtypes('float')

#Clusterizacao
kmeans = KMeans(n_clusters=2).fit(X)

#Labels preditos pelo kmeans
X['label'] = kmeans.predict(X)

#Pairplot
sns.pairplot(data=X,vars=['PC4','PC5'],hue='label')
plt.show()

#Pairplot
sns.pairplot(data=df_parkinson,vars=['PC4','PC5'],hue='drug')
plt.show()

#Swap
X['label'] = X['label'].map({0:1,1:0})

(X['label'] == df_parkinson['drug']).sum() / X.shape[0]